# Language Classification

## Sentiment Analysis
- Raw text is tokenized.
- Feed token IDs into model (Into first, Embedding layer which then translates into world vectors).
- Output activations -> Transformer heads -> Probabilities for each possible response like positive, negative and neutral, which all adds up to 1 (Softmax).
- argmax (Looks at each value and return the index of the greatest).

### Install Dependencies

In [1]:
!pip install -U pip

In [2]:
!pip install -U openssl

ERROR: Could not find a version that satisfies the requirement openssl (from versions: none)
ERROR: No matching distribution found for openssl


In [3]:
!pip install -U flair

### Import Dependencies

In [4]:
import flair

### Disabling SSL Verification for Model Downloads

In [5]:
import warnings
import contextlib

import requests
from urllib3.exceptions import InsecureRequestWarning

old_merge_environment_settings = requests.Session.merge_environment_settings

@contextlib.contextmanager
def no_ssl_verification():
    opened_adapters = set()

    def merge_environment_settings(self, url, proxies, stream, verify, cert):
        # Verification happens only once per connection so we need to close
        # all the opened adapters once we're done. Otherwise, the effects of
        # verify=False persist beyond the end of this context manager.
        opened_adapters.add(self.get_adapter(url))

        settings = old_merge_environment_settings(self, url, proxies, stream, verify, cert)
        settings['verify'] = False

        return settings

    requests.Session.merge_environment_settings = merge_environment_settings

    try:
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', InsecureRequestWarning)
            yield
    finally:
        requests.Session.merge_environment_settings = old_merge_environment_settings

        for adapter in opened_adapters:
            try:
                adapter.close()
            except:
                pass

### Code

In [7]:
# 1. Initialize the model.
model = None
with no_ssl_verification():
    model = flair.models.TextClassifier.load('en-sentiment')

2022-03-21 12:04:18,505 loading file C:\Users\divanma\.flair\models\sentiment-en-mix-distillbert_4.pt
Model Ready!


In [21]:
# 2. Tokenize the input text.
text = 'This is absolutely amazing!'
sentence = flair.data.Sentence(text)

# 3. Process the token ids with the model.
model.predict(sentence)

# 4. Format the outputs.
label = sentence.get_labels()[0]
predicted_type = label.value
probability = label.score

print(f'"{text}" predicted as {predicted_type} with a confidence of {probability}.')

"This is absolutely amazing!" predicted as POSITIVE with a confidence of 0.9989346861839294.


# References
- https://github.com/flairNLP/flair
- https://curatedpython.com/p/a-very-flairnlp-flair/index.html